## Merge Spdl

In [2]:
# # Associate Ca2+ signal with spindles for each session & subsessions using crossregistration

#######################################################################################
                            # Define Experiment type #
#######################################################################################

#DrugExperiment=0 #if Baseline Experiment =1#if CGP Experiment
DrugExperiment=1

Method=0 # 1=AB 0=AH

#######################################################################################
                                # Load packages #
#######################################################################################

import os
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy.interpolate import interp2d
from scipy.signal import find_peaks
from scipy.stats import zscore
import pickle
import os
from scipy.interpolate import griddata
import logging
import sys 
import shutil
from bisect import bisect_left
from ast import literal_eval

from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer
from IPython.display import display
from ipyfilechooser import FileChooser
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

#######################################################################################
                                # Define functions #
#######################################################################################
def is_overlapping(starttime, endtime, starttimeList, endtimeList):
    IsTrue='False'
    for ind in starttimeList.index: #range(len(starttimeList)):
        if starttime<=starttimeList[ind] and starttimeList[ind]<=endtime: # event n°2 begins after the start n°1               
            if (endtime-starttimeList[ind])>=int(0.5*(endtime-starttime)): # overlapp > to 50% of the duration of the event n°1
                IsTrue='True'
                break                
        elif starttime<=endtimeList[ind] and endtimeList[ind]<=endtime: # event n°2 ends before the end n°1 
            if (endtimeList[ind]-starttime)>=int(0.5*(endtime-starttime)): # overlapp > to 50% of the duration of the event n°1
                IsTrue='True'
                break
    return IsTrue, ind


def find_session_folders(root_path):
    sessions = []
    sessions_path=[]
    # Iterate through items in the root_path
    for item in os.listdir(root_path):
        item_path = os.path.join(root_path, item)
        if os.path.isdir(item_path):
            # Check if the directory name contains "session"
            if "session" in item:
                sessions.append(item)
                sessions_path.append(item_path)
            else:
                # Check the subdirectories of the current directory
                for sub_item in os.listdir(item_path):
                    sub_item_path = os.path.join(item_path, sub_item)
                    if os.path.isdir(sub_item_path) and "session" in sub_item:
                        sessions.append(sub_item)
                        sessions_path.append(sub_item_path)
                        
    return sessions, sessions_path

def restriction_parameter(All_DS):
    nb_spindle = All_DS.shape[0]
    listtodrop = []
    for tt in range(nb_spindle-1):
        # merge spdl that starts within a spdl
        if(All_DS['end time'][tt]>All_Spindle['start time'][tt + 1]):
            if(All_Spindle['Duration'][tt]<All_Spindle['Duration'][tt + 1]):
                if(All_Spindle['start time'][tt]<All_Spindle['start time'][tt + 1]):
                    All_Spindle['start time'][tt+1] = All_Spindle['start time'][tt]
                    listtodrop.append(tt)
                else:
                    listtodrop.append(tt)
            if(All_Spindle['Duration'][tt]>All_Spindle['Duration'][tt + 1]):
                if(All_Spindle['end time'][tt]<All_Spindle['end time'][tt + 1]):
                    All_Spindle['end time'][tt] = All_Spindle['end time'][tt + 1]
                    listtodrop.append(tt+1)
                else:
                    listtodrop.append(tt+1)

    for tt in range(nb_spindle-1):
        # merge spdls that are 200ms apart
        if((All_Spindle['start time'][tt + 1] - All_Spindle['end time'][tt])<200):
            if((All_Spindle['Duration'][tt])<All_Spindle['Duration'][tt + 1]): #first spdl longer so remove/merge the second one
                All_Spindle['start time'][tt + 1] = All_Spindle['start time'][tt]
                listtodrop.append(tt)
            if((All_Spindle['Duration'][tt+1])<All_Spindle['Duration'][tt]): #second spdl longer so remove/merge the first one
                All_Spindle['end time'][tt] = All_Spindle['start time'][tt + 1]
                listtodrop.append(tt+1)

    for tt in range(nb_spindle):
        #Update duration because of the merging
        All_Spindle['Duration'][tt]=All_Spindle['end time'][tt]-All_Spindle['start time'][tt]

    for tt in range(nb_spindle): #All_Spindle.index:
        #Remove Spdl that last less than 500ms
        if (All_Spindle['Duration'][tt]<500):
            listtodrop.append(tt)        
    
    All_Spindle = All_Spindle.drop(listtodrop) 
    All_Spindle = All_Spindle.reset_index(drop=True)
    return All_Spindle

    
#######################################################################################
                # Load sleep score and Ca2+ time series numpy arrays #
#######################################################################################

MiceList=['BlackLinesOK', 'BlueLinesOK', 'GreenDotsOK','Purple' ,'ThreeColDotsOK'] if DrugExperiment else ['BlackLinesOK', 'BlueLinesOK', 'GreenDotsOK', 'GreenLinesOK', 'Purple', 'RedLinesOK','ThreeColDotsOK', 'ThreeBlueCrossesOK']

for mice in MiceList:

    dpath0 = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/" if DrugExperiment else "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/"
    dpath=dpath0 + mice
    print(f"####################################################################################")
    print(f"################################### {mice} ####################################")
    print(f"####################################################################################")
    print(f"Path to the folder : {dpath}")
    folder_base = Path(dpath)

    sessions, sessions_path = find_session_folders(folder_base)
    nb_sessions=len(sessions)

    for sess,session in enumerate(sessions):  
        session_path=Path(sessions_path[sess])
        Spindleproperties_PFC = session_path / f'OpenEphys/Spindlesproperties_PFC_7sd_AB.xlsx' if Method else session_path / f'OpenEphys/Spindleproperties_PFC.csv'
        Spindleproperties_S1 = session_path / f'OpenEphys/Spindlesproperties_S1_7sd_AB.xlsx' if Method else session_path / f'OpenEphys/Spindleproperties_S1.csv'

        PFClist = pd.read_excel(Spindleproperties_PFC) if Method else pd.read_csv(Spindleproperties_PFC)
        if DrugExperiment: 
            PFClist['toKeep'] = PFClist['toKeep'].astype(str)
            PFClist  = PFClist[PFClist['toKeep'].isin(['VRAI', 'True'])]
        else: 
            PFClist['toKeep'] ='VRAI'
        PFClist['CTX']='PFC'
        PFClist['StartingLoc']='PFC'
        PFClist = PFClist.reset_index(drop=True)
        NewPFClist=restriction_parameter(PFClist)

        S1list  = pd.read_excel(Spindleproperties_S1) if Method else pd.read_csv(Spindleproperties_S1)   
        if DrugExperiment: 
            S1list['toKeep'] = S1list['toKeep'].astype(str)
            S1list  = S1list[S1list['toKeep'].isin(['VRAI', 'True'])]
        else: 
            S1list['toKeep'] ='VRAI'
        S1list['CTX']='S1'
        S1list['StartingLoc']='S1'
        S1list = S1list.reset_index(drop=True)
        NewS1list=restriction_parameter(S1list)

        Spdllist=pd.concat([NewPFClist,NewS1list],ignore_index=True)  
        Spdllist = Spdllist.drop(Spdllist.columns[0], axis=1)  

        Spdllist['LocalGlobal']='Local'

        Spdllist = Spdllist.sort_values(by='start time')
        Spdllist = Spdllist.reset_index(drop=True)

        liststarts=Spdllist["start time"]
        listends=Spdllist["end time"]
        NewSpdllist=Spdllist.copy()

        for spdl1 in NewSpdllist.index : # range(len(Spdllist)) :
            start1=liststarts[spdl1]
            end1=listends[spdl1]
            otherunit_range = [x for x in NewSpdllist.index  if x != spdl1]
            #print(f'spdl1 n°{spdl1}')
            if NewSpdllist.loc[spdl1,'toKeep'] != 'False':
                for spdl2 in otherunit_range:
                    if NewSpdllist.loc[spdl2,'toKeep'] != 'False':
                        start2=liststarts[spdl2]
                        end2=listends[spdl2]
                        if start1<=start2 and start2<=end1: # event n°2 begins after the start n°1               
                            if (end1-start2)>=int(0.5*(end1-start1)): # overlapp > to 50% of the duration of the event n°1
                                if NewSpdllist.loc[spdl1,'CTX']!= NewSpdllist.loc[spdl2,'CTX']:
                                    NewSpdllist.loc[spdl1, 'LocalGlobal']='Global'
                                    NewSpdllist.loc[spdl1, 'StartingLoc']=NewSpdllist.loc[spdl1,'CTX']
                                    NewSpdllist.loc[spdl1, 'CTX']='S1PFC'
                                    NewSpdllist.loc[spdl1, 'start time']=int((start1 + start2)/2)
                                    NewSpdllist.loc[spdl1, 'end time']=max(end1, end2)
                                    NewSpdllist.loc[spdl1, 'Duration']=max(end1, end2)-int((start1 + start2)/2)
                                    NewSpdllist.loc[spdl2, 'toKeep']='False'
                                    #print(f'Cdt n°1: Global, keep spdl n°{spdl1} {start1} instead of spdl n°{spdl2} {start2}')
                                    break
                        elif start1<=end2 and end2<=end1: # event n°2 ends before the end n°1 
                            if (end2-start1)>=int(0.5*(end1-start1)): # overlapp > to 50% of the duration of the event n°1
                                if NewSpdllist.loc[spdl1,'CTX']!= NewSpdllist.loc[spdl2,'CTX']:
                                    NewSpdllist.loc[spdl2, 'LocalGlobal']='Global'
                                    NewSpdllist.loc[spdl2, 'StartingLoc']=NewSpdllist.loc[spdl2,'CTX']
                                    NewSpdllist.loc[spdl2, 'CTX']='S1PFC'
                                    NewSpdllist.loc[spdl2, 'start time']=int((start1 + start2)/2)
                                    NewSpdllist.loc[spdl2, 'end time']=max(end1, end2)
                                    NewSpdllist.loc[spdl2, 'Duration']=max(end1, end2)-int((start1 + start2)/2)
                                    NewSpdllist.loc[spdl1, 'toKeep']='False'
                                    #print(f'Cdt n°2: Global, keep spdl n°{spdl2} {start2}, instead of spdl n°{spdl1} {start1}')
                                    break
        NewSpdllist = NewSpdllist[NewSpdllist['toKeep'].isin(['True', 'VRAI'])]
        NewSpdllist = NewSpdllist.sort_values(by='start time')
        NewSpdllist = NewSpdllist.reset_index(drop=True)
        NewSpdllist=restriction_parameter(NewSpdllist)

        filenameOutput = session_path / f'OpenEphys/Spindlesproperties_S1&PFC_7sd_AB.xlsx'  if Method else session_path / f'OpenEphys/Spindleproperties_S1&PFC.csv'
        if Method:
            NewSpdllist.to_excel(filenameOutput)
        else:
            NewSpdllist.to_csv(filenameOutput, sep= ',')



####################################################################################
################################### BlackLinesOK ####################################
####################################################################################
Path to the folder : //10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/BlackLinesOK
####################################################################################
################################### BlueLinesOK ####################################
####################################################################################
Path to the folder : //10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/BlueLinesOK
####################################################################################
################################### GreenDotsOK ####################################
####################################################################################
Path to

## Merge DS

In [14]:
# # Associate Ca2+ signal with spindles for each session & subsessions using crossregistration

#######################################################################################
                            # Define Experiment type #
#######################################################################################

#DrugExperiment=0 #if Baseline Experiment =1#if CGP Experiment
DrugExperiment=0

#######################################################################################
                                # Load packages #
#######################################################################################

import os
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy.interpolate import interp2d
from scipy.signal import find_peaks
from scipy.stats import zscore
import pickle
import os
from scipy.interpolate import griddata
import logging
import sys 
import shutil
from bisect import bisect_left
from ast import literal_eval

from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer
from IPython.display import display
from ipyfilechooser import FileChooser
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

#######################################################################################
                                # Define functions #
#######################################################################################
def is_overlapping(starttime, endtime, starttimeList, endtimeList):
    IsTrue='False'
    for ind in starttimeList.index: #range(len(starttimeList)):
        if starttime<=starttimeList[ind] and starttimeList[ind]<=endtime: # event n°2 begins after the start n°1               
            if (endtime-starttimeList[ind])>=int(0.5*(endtime-starttime)): # overlapp > to 50% of the duration of the event n°1
                IsTrue='True'
                break                
        elif starttime<=endtimeList[ind] and endtimeList[ind]<=endtime: # event n°2 ends before the end n°1 
            if (endtimeList[ind]-starttime)>=int(0.5*(endtime-starttime)): # overlapp > to 50% of the duration of the event n°1
                IsTrue='True'
                break
    return IsTrue, ind


def find_session_folders(root_path):
    sessions = []
    sessions_path=[]
    # Iterate through items in the root_path
    for item in os.listdir(root_path):
        item_path = os.path.join(root_path, item)
        if os.path.isdir(item_path):
            # Check if the directory name contains "session"
            if "session" in item:
                sessions.append(item)
                sessions_path.append(item_path)
            else:
                # Check the subdirectories of the current directory
                for sub_item in os.listdir(item_path):
                    sub_item_path = os.path.join(item_path, sub_item)
                    if os.path.isdir(sub_item_path) and "session" in sub_item:
                        sessions.append(sub_item)
                        sessions_path.append(sub_item_path)
                        
    return sessions, sessions_path

def restriction_parameter(All_DS):
    nb_ds = All_DS.shape[0]
    listtodrop = []
    for tt in range(nb_ds): #All_DS.index:
        #Remove DS that last less than 500ms
        if (All_DS['Duration'][tt]>500):
            listtodrop.append(tt)        
    
    All_DS = All_DS.drop(listtodrop) 
    All_DS = All_DS.reset_index(drop=True)
    return All_DS

    
#######################################################################################
                # Load sleep score and Ca2+ time series numpy arrays #
#######################################################################################

MiceList=['BlackLinesOK', 'BlueLinesOK', 'GreenDotsOK','Purple' ,'ThreeColDotsOK'] if DrugExperiment else ['BlackLinesOK', 'BlueLinesOK', 'GreenDotsOK', 'GreenLinesOK', 'Purple', 'RedLinesOK','ThreeColDotsOK', 'ThreeBlueCrossesOK']

for mice in MiceList:

    dpath0 = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/CGP/" if DrugExperiment else "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/"
    dpath=dpath0 + mice
    print(f"####################################################################################")
    print(f"################################### {mice} ####################################")
    print(f"####################################################################################")
    print(f"Path to the folder : {dpath}")
    folder_base = Path(dpath)

    sessions, sessions_path = find_session_folders(folder_base)
    nb_sessions=len(sessions)

    for sess,session in enumerate(sessions):  
        session_path=Path(sessions_path[sess])
        DSproperties_S1 = session_path / f'OpenEphys/DownStatesproperties_S1_2Pro3Height_AB.xlsx' 
        DSproperties_PFC = session_path / f'OpenEphys/DownStatesproperties_PFC_2Pro3Height_AB.xlsx' 

        PFClist = pd.read_excel(DSproperties_PFC) 
        #if DrugExperiment: 
        #    PFClist['toKeep'] = PFClist['toKeep'].astype(str)
        #    PFClist  = PFClist[PFClist['toKeep'].isin(['VRAI', 'True'])]
        #else: 
        PFClist['toKeep'] ='VRAI'
        PFClist['CTX']='PFC'
        PFClist['StartingLoc']='PFC'
        PFClist = PFClist.reset_index(drop=True)
        NewPFClist=restriction_parameter(PFClist)

        S1list  = pd.read_excel(DSproperties_S1)  
        #if DrugExperiment: 
        #    S1list['toKeep'] = S1list['toKeep'].astype(str)
        #    S1list  = S1list[S1list['toKeep'].isin(['VRAI', 'True'])]
        #else: 
        S1list['toKeep'] ='VRAI'
        S1list['CTX']='S1'
        S1list['StartingLoc']='S1'
        S1list = S1list.reset_index(drop=True)
        NewS1list=restriction_parameter(S1list)

        DSlist=pd.concat([NewPFClist,NewS1list],ignore_index=True)  
        DSlist = DSlist.drop(DSlist.columns[0], axis=1)  

        DSlist['LocalGlobal']='Local'

        DSlist = DSlist.sort_values(by='start time')
        DSlist = DSlist.reset_index(drop=True)

        liststarts=DSlist["start time"]
        listends=DSlist["end time"]
        NewDSlist=DSlist.copy()

        for ds1 in NewDSlist.index : 
            start1=liststarts[ds1]
            end1=listends[ds1]
            otherunit_range = [x for x in NewDSlist.index  if x != ds1]
            #print(f'spdl1 n°{spdl1}')
            if NewDSlist.loc[ds1,'toKeep'] != 'False':
                for ds2 in otherunit_range:
                    if NewDSlist.loc[ds2,'toKeep'] != 'False':
                        start2=liststarts[ds2]
                        end2=listends[ds2]
                        if start1<=start2 and start2<=end1: # event n°2 begins after the start n°1               
                            if (end1-start2)>=int(0.5*(end1-start1)): # overlapp > to 50% of the duration of the event n°1
                                if NewDSlist.loc[ds1,'CTX']!= NewDSlist.loc[ds2,'CTX']:
                                    NewDSlist.loc[ds1, 'LocalGlobal']='Global'
                                    NewDSlist.loc[ds1, 'StartingLoc']=NewDSlist.loc[ds1,'CTX']
                                    NewDSlist.loc[ds1, 'CTX']='S1PFC'
                                    NewDSlist.loc[ds1, 'start time']=int((start1 + start2)/2)
                                    NewDSlist.loc[ds1, 'end time']=max(end1, end2)
                                    NewDSlist.loc[ds1, 'Duration']=max(end1, end2)-int((start1 + start2)/2)
                                    NewDSlist.loc[ds2, 'toKeep']='False'
                                    break
                        elif start1<=end2 and end2<=end1: # event n°2 ends before the end n°1 
                            if (end2-start1)>=int(0.5*(end1-start1)): # overlapp > to 50% of the duration of the event n°1
                                if NewDSlist.loc[ds1,'CTX']!= NewDSlist.loc[ds2,'CTX']:
                                    NewDSlist.loc[ds2, 'LocalGlobal']='Global'
                                    NewDSlist.loc[ds2, 'StartingLoc']=NewDSlist.loc[ds2,'CTX']
                                    NewDSlist.loc[ds2, 'CTX']='S1PFC'
                                    NewDSlist.loc[ds2, 'start time']=int((start1 + start2)/2)
                                    NewDSlist.loc[ds2, 'end time']=max(end1, end2)
                                    NewDSlist.loc[ds2, 'Duration']=max(end1, end2)-int((start1 + start2)/2)
                                    NewDSlist.loc[ds1, 'toKeep']='False'
                                    break
        NewDSlist = NewDSlist[NewDSlist['toKeep'].isin(['True', 'VRAI'])]
        NewDSlist = NewDSlist.sort_values(by='start time')
        NewDSlist = NewDSlist.reset_index(drop=True)
        NewDSlist=restriction_parameter(NewDSlist)

        filenameOutput = session_path / f'OpenEphys/DownStatesproperties_S1&PFC.csv'
        NewDSlist.to_csv(filenameOutput, sep= ',')



####################################################################################
################################### BlackLinesOK ####################################
####################################################################################
Path to the folder : //10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/BlackLinesOK
####################################################################################
################################### BlueLinesOK ####################################
####################################################################################
Path to the folder : //10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/BlueLinesOK
####################################################################################
################################### GreenDotsOK ####################################
########################################